In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('.')


In [2]:
from load_miller import split_classes, continues_classes, continues_classes_idle

# Load data

In [3]:
import numpy as np
fname = 'motor_imagery.npz'
data = np.load(fname, allow_pickle=True)['dat']

In [11]:
data[0][0].keys()

dict_keys(['t_off', 'stim_id', 't_on', 'srate', 'V', 'scale_uv', 'locs', 'hemisphere', 'lobe', 'gyrus', 'Brodmann_Area'])

In [4]:
a = np.zeros((10,10))
len(a.shape)

2

In [5]:
for i in range(7):
    for j in range(2):
        print(data[i][j]['V'].shape[0], data[i][j]['t_off'][-1], data[i][j]['t_on'][0], data[i][j]['t_off'][-1] - data[i][j]['t_on'][0] + 1)

376400 369440 10160 359281
376600 369640 10160 359481
390680 383720 10120 373601
390320 383360 10160 373201
376240 369280 10120 359161
376840 369880 10160 359721
390720 383760 10120 373641
390200 383240 10120 373121
390640 383680 10200 373481
390160 383200 10200 373001
390720 383760 10120 373641
390200 383240 10120 373121
390240 383280 10120 373161
390920 383960 10120 373841


In [8]:
390920  - 376240

14680

In [10]:
(390920  - 376240) // 2

7340

In [ ]:
data[0][0]['stim_id']

In [ ]:
data[0][0]['V'].shape

In [ ]:
np.zeros(data[0][0]['V'][:,:46].shape).shape

In [ ]:
data[0][0]['t_on'].shape[0]

In [7]:
dat1 = data[1][0] 

discrete_data, extra_info_dat1 = split_classes(data=dat1)

continues_data_1_dat1, extra_info_1_dat1 = continues_classes(data=dat1)

continues_data_2_dat1, extra_info_2_dat1 = continues_classes_idle(data=dat1)

In [9]:
continues_data_1_dat1['stim_id'].shape

(60, 3000, 46)

In [37]:
a = [[11,11,12,11,11,12,12,12,11] , [1,1,2,1,1,2,11,2,13]]
a = np.array(a)

# (a == 12).astype(int)
a = a+2
a[a == 3] = 1
a

array([[13, 13, 14, 13, 13, 14, 14, 14, 13],
       [ 1,  1,  4,  1,  1,  4, 13,  4, 15]])

In [12]:
dat2 = data[2][0] 

tongue_data_dat2, hand_data_dat2, extra_info_dat2 = split_classes(data=dat2)

continues_data_1_dat2, extra_info_1_dat2 = continues_classes(data=dat2)

continues_data_2_dat2, extra_info_2_dat2 = continues_classes_idle(data=dat2)

In [13]:
max_iterations = 10000
output_dimension = 3

from cebra import CEBRA

cebra_posdir3_model = CEBRA(model_architecture='offset10-model',
                        batch_size=64,
                        learning_rate=3e-4,
                        temperature=1,
                        output_dimension=output_dimension,
                        max_iterations=max_iterations,
                        distance='cosine',
                        conditional='time_delta',
                        device='cuda_if_available',
                        verbose=True,
                        time_offsets=10)


In [ ]:
cebra_posdir3_model.fit(continues_data_2_dat1['V'], continues_data_2_dat1['stim_id'])
cebra_posdir3 = cebra_posdir3_model.transform(continues_data_2_dat1['V'])

In [ ]:
cebra_posdir3.shape

In [ ]:
continues_data_2_dat1['stim_id'].shape

In [ ]:
cebra_posdir3.shape

In [ ]:
# import numpy as np
# import plotly.express as px

# # Example data
# # Replace these with your actual data
# points = cebra_posdir3  # Shape: (390680, 3)
# labels = continues_data_2_dat1['stim_id'][:,0]  # Shape: (390680,)

# # Create a DataFrame for easier handling (optional)
# import pandas as pd
# df = pd.DataFrame(points, columns=["X", "Y", "Z"])
# df["Label"] = labels

# # Create a dynamic 3D scatter plot
# fig = px.scatter_3d(
#     df,
#     x="X", 
#     y="Y", 
#     z="Z", 
#     color="Label",  # Color by labels
#     color_continuous_scale="rainbow",
#     title="Dynamic 3D Scatter Plot",
#     labels={"Label": "Classes"},
#     opacity=0.8
# )

# # Show the plot
# fig.show()


In [ ]:
# import numpy as np
# import plotly.express as px

# # Example data
# # Replace these with your actual data
# points = cebra_posdir3  # Shape: (390680, 3)
# labels = continues_data_2_dat1['stim_id'][:,0]  # Shape: (390680,)

# # Create a DataFrame for easier handling (optional)
# import pandas as pd
# df = pd.DataFrame(points, columns=["X", "Y", "Z"])
# df["Label"] = labels

# # Create a dynamic 3D scatter plot
# fig = px.scatter_3d(
#     df,
#     x="X", 
#     y="Y", 
#     z="Z", 
#     color="Label",  # Color by labels
#     color_continuous_scale="rainbow",
#     title="Dynamic 3D Scatter Plot",
#     labels={"Label": "Classes"},
#     opacity=0.8
# )

# fig.update_traces(marker=dict(size=2))

# # Show the plot
# fig.show()
